#### To Submit results here: https://courses.datatalks.club/llm-zoomcamp-2024/homework/hw1


In [1]:
import requests 
from openai import OpenAI
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken

/home/codespace/.local/share/virtualenvs/Workshop-1-QA-Bot-d4ZJ_5na/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Q1. Running Elastic

Run Elastic Search 8.4.3, and get the cluster information. If you run it on localhost, this is how you do it:

```bash
curl localhost:9200
```

What's the `version.build_hash` value?

In [17]:
! curl localhost:9200 | jq '.version.build_hash'


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   539  100   539    0     0  59888      0 --:--:-- --:--:-- --:--:-- 67375
"42f05b9372a9a4a470db3b52817899b99a76ee73"


### Answer 1: 42f05b9372a9a4a470db3b52817899b99a76ee73

## Getting the data

Now let's get the FAQ data. You can run this snippet:

```python
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
```

Note that you need to have the `requests` library:

```bash
pip install requests
```

In [20]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the `course` field a keyword and the rest should be text. 

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* `insert`
* `index`
* `put`
* `add`

In [5]:
es_client = Elasticsearch('http://localhost:9200') 

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

In [7]:
index_name = "very-good-index"

In [ ]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'very-good-index'})

In [33]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████| 948/948 [00:24<00:00, 39.50it/s]


In [35]:
doc

{'text': 'Problem description\nInfrastructure created in AWS with CD-Deploy Action needs to be destroyed\nSolution description\nFrom local:\nterraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" --reconfigure\nterraform destroy --var-file vars/prod.tfvars\nAdded by Erick Calderin',
 'section': 'Module 6: Best practices',
 'question': 'How to destroy infrastructure created via GitHub Actions',
 'course': 'mlops-zoomcamp'}

### Answer 2: index

## Q3. Searching

Now let's search in our index. 

For a query "How do I execute a command in a running docker container?", what's the score for the top ranking result? 

Use only `question` and `text` fields and give `question` a boost of 4

* 94.05
* 84.05
* 74.05
* 64.05

Look at the `_score` field.

In [3]:
def elastic_search(query, size, filter_condition = {"filter":{}}):
    must_condition = {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                                   }
                       }
                    }
    if filter_condition["filter"] != {}:
        must_condition.update(filter_condition)
    
    search_query = {
        "size": size,
        "query": {
            "bool": {
                **must_condition,
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append({'score': hit['_score'], 'source': hit['_source']})
    
    return result_docs

In [8]:
query = "How do I execute a command in a running docker container?"

results = elastic_search(query, size=1)

In [9]:
results

[{'score': 84.050095,
  'source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}}]

### Answer 3: The score is 84.05

## Q4. Filtering

Now let's only limit the questions to `machine-learning-zoomcamp`.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [11]:
results = elastic_search(query, size=3, filter_condition = {"filter": {"term": {"course": "machine-learning-zoomcamp"}}})
results

[{'score': 84.050095,
  'source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'score': 51.04628,
  'source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
   'section': '5. Deploying Machine Learning Models',
   'questio

#### Answer 4: How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM. Let's use these templates:


```python
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
```

Separate context entries by two linebreaks (`\n\n`)

Put the data from Q4 there. 

What's the length of the result? (use the `len` function)

* 962
* 1462
* 1962
* 2462

In [12]:
search_results = [res['source'] for res in results]

In [13]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [14]:
prompt = build_prompt(query, search_results)
print(len(prompt))

1462


#### Answer 5: 1462

## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of 
tokens we send in our prompt and receive in the response.

The OpenAI python package uses `tiktoken` for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query: 

```python
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the `encode` function. How many tokens does our prompt have?

* 122
* 222
* 322 
* 422

Note: to decode back a token into a word, you can use the `decode_single_token_bytes` function:

```python
encoding.decode_single_token_bytes(63842)
```

In [15]:
encoding = tiktoken.encoding_for_model("gpt-4o")


In [17]:
len(encoding.encode(prompt))

322

#### Answer 6: 422

## Bonus: generating the answer (ungraded)

Let's send the prompt to OpenAI. What's the response?  

In [18]:
client = OpenAI()

In [19]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [26]:
def rag(query):
    results = elastic_search(query, size=3, filter_condition = {"filter": {"term": {"course": "machine-learning-zoomcamp"}}})
    search_results = [res['source'] for res in results]
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [27]:
rag(query)

'To execute a command in a running docker container, you can use the following steps:\n\n1. First, find the container ID of the running container by using the `docker ps` command.\n\n2. Then, execute the desired command in the specific container using the `docker exec` command. For example, if you want to start a bash session inside the container, you can use:\n   ```\n   docker exec -it <container-id> bash\n   ```\n\nHere is the process in brief:\n\n```sh\ndocker ps  # This will list all running containers and their IDs\ndocker exec -it <container-id> bash  # Replace <container-id> with the actual container ID from the previous step\n```'